In [1]:
import numpy as np
import pandas as pd
from optbinning import OptimalBinning

/home/huangdan/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/huangdan/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


(CVXPY) Feb 08 10:42:13 PM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Unrecognized new version of ortools (9.8.3296). Expected < 9.8.0. Please open a feature request on cvxpy to enable support for this version.')
(CVXPY) Feb 08 10:42:13 PM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Unrecognized new version of ortools (9.8.3296). Expected < 9.8.0. Please open a feature request on cvxpy to enable support for this version.')


In [2]:
file_path = "/hdd1/sams/census/census.arff"
with open(file_path, 'r') as f:
    lines = f.readlines()

In [6]:
flag = False
rows = []
for line in lines:
    if line.strip() == "@data":
        flag = True
        continue
    
    if not flag:
        continue
    
    row = [i for i in line.strip().split(',')]
    if len(row) != 42:
        print(row)
        break
    rows.append(row) 
del lines

In [7]:
df = pd.DataFrame(rows)

In [9]:
numerical_cols = [0,2,3,5,16,17,18,24,30,36,38,39,40]
len(numerical_cols)

13

In [11]:
bin_cols = []
print(f"#col: {len(df.columns)}")
for col in numerical_cols:
    n = len(df[col].unique())
    if n > 7:
        print(f"{col}:{n}")
        bin_cols.append(col)

#col: 42
0:91
2:52
3:47
5:1425
16:133
17:114
18:1675
24:123232
39:53


In [12]:
# transfer these bin cols to float
for col in bin_cols:
    try:
        df[col] = df[col].astype(float)
    except:
        print(col)

In [15]:
print(df[41].unique())

["' - 50000.'" "' 50000+.'"]


In [17]:
# label to 1,0
map_y = {"' - 50000.'": 0, "' 50000+.'":1}
df[41] = df[41].map(map_y)

In [19]:
print(f"POS/ALL : {sum(df[41].values)}/{len(df)}")

POS/ALL : 18568/299285


In [20]:
Y = df[41]
del df[41]
# X = df[range(69)]
X = df

In [21]:
def optbinning_data(df:pd.DataFrame, column:int, y:np.array):
    x = df[column].values

    optb = OptimalBinning(name="23", dtype="numerical", solver="cp",
                      monotonic_trend="auto", 
                      max_n_prebins=100,
                      min_prebin_size=0.001, time_limit=200)
    optb.fit(x,y)
    res = optb.transform(x, metric="indices")
    bin_number = max(res)
    status = optb.status
    set_n = len(df[column].unique())
    print(f"bin column:{column}, set size {set_n}, bin status {status}, bin_number {bin_number}")
    return res

In [22]:
for col in bin_cols:
    v = optbinning_data(X, col, Y.values)
    X[col] = v 

bin column:0, set size 91, bin status OPTIMAL, bin_number 45
bin column:2, set size 52, bin status OPTIMAL, bin_number 5
bin column:3, set size 47, bin status OPTIMAL, bin_number 1
bin column:5, set size 1425, bin status OPTIMAL, bin_number 5
bin column:16, set size 133, bin status OPTIMAL, bin_number 6
bin column:17, set size 114, bin status OPTIMAL, bin_number 6
bin column:18, set size 1675, bin status OPTIMAL, bin_number 16
bin column:24, set size 123232, bin status FEASIBLE, bin_number 10
bin column:39, set size 53, bin status OPTIMAL, bin_number 18


In [34]:
# tokenization
col_to_value2token = {}
cnt = 0
for col in X.columns:
    col_values = X[col].values
    col_to_value2token[col] = {}
    col_values_set = set(col_values)
    for val in col_values_set:
        if val not in col_to_value2token[col]:
            col_to_value2token[col][val] = cnt
            cnt += 1
print(cnt)

532


In [35]:
# replace value with token
for col in X.columns:
    value_to_token = col_to_value2token[col]
    X[col] = X[col].map(value_to_token)

In [36]:
from sklearn.model_selection import train_test_split

X_train, X_tmp, Y_train, Y_tmp = train_test_split(X,Y, test_size=0.2, random_state=2024)
X_val, X_test,Y_val, Y_test = train_test_split(X_tmp, Y_tmp, test_size=0.5, random_state=2024)

In [37]:
# save to libsvm
def save_dataset(X, Y, file_path:str):
    with open(file_path, 'w', encoding='utf-8') as f:
        n = len(X)
        for i in range(n):
            row = X.iloc[i].values 
            y = Y.iloc[i]
            items =[str(y)] + [str(i)+":1" for i in row]
            line = ' '.join(items) + '\n'
            f.write(line)

In [38]:
train_file = '/hdd1/sams/data/census/train.libsvm'
val_file = '/hdd1/sams/data/census/val.libsvm'
test_file = '/hdd1/sams/data/census/test.libsvm'
save_dataset(X_train, Y_train, train_file)
save_dataset(X_val, Y_val, val_file)
save_dataset(X_test, Y_test, test_file)

In [39]:
print(f"train: POS/ALL -> {sum(Y_train.values.reshape(-1))}/{len(Y_train)}")
print(f"val: POS/ALL -> {sum(Y_val.values.reshape(-1))}/{len(Y_val)}")
print(f"test: POS/ALL -> {sum(Y_test.values.reshape(-1))}/{len(Y_test)}")

train: POS/ALL -> 14804/239428
val: POS/ALL -> 1835/29928
test: POS/ALL -> 1929/29929


In [40]:
print(f"columns:{len(X_train.columns)}")

columns:41
